<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/biz/profitViewSMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, time
from google.colab import userdata

### Change to your ProfitView webhook id
RESTFUL_ID = userdata.get('profitView')
BASE_URL = 'https://profitview.net/trading/signal/{}/'.format(RESTFUL_ID)


# Get historical candle data
def getHistData(field, num, level="1m"):
  histData = []

  url = BASE_URL + 'candles?level={}'.format(level)
  r = requests.get(url)
  candleData = r.json()['data']['data']

  for data in candleData[:5]:
    histData.append(data[field])

  return histData


# Get the latest trade price
def getTradePx():
  url = BASE_URL + 'trade'
  trade = requests.get(url).json()

  return trade['data']['price']


def calculateSMA(tradePxList, shortMins):
  smaLong = sum(tradePxList) / len(tradePxList)
  smaShort = sum(tradePxList[:shortMins]) / len(tradePxList[:shortMins])

  return smaLong, smaShort


def updateTradePxList(tradePxList, tradePx):
    tradePxList.insert(0, tradePx)
    tradePxList = tradePxList[:len(tradePxList)-1]

    return tradePxList

# 1: long signal
# 0: flat signal
# -1: short signal
def getTradingSignal(smaLong, smaShort):
  if smaShort > smaLong: return 1
  elif smaShort < smaLong: return -1

  return 0

In [ ]:
# Init trading parameters
WAIT_SECOND = 3
SMA_SHORT_MINS = 2
SMA_LONG_MINS = 5

In [ ]:
# Calculate trading indicators from the historical data
tradePxList = getHistData('close', SMA_LONG_MINS)
smaLong, smaShort = calculateSMA(tradePxList, SMA_SHORT_MINS)

In [ ]:
# For each market event (quote/trade update) or timer event
while True:
    # Update trading indicators from the current market data
    tradePxList = updateTradePxList(tradePxList, getTradePx())

    # Calculate trading indicators from the current market data
    smaLong, smaShort = calculateSMA(tradePxList, SMA_SHORT_MINS)

    # Determine the trading signal
    signal = getTradingSignal(smaLong, smaShort)
    print("smaLong={}, smaShort={}, signal={}".format(smaLong, smaShort, signal))

    # Determine the action

    time.sleep(WAIT_SECOND)